This is just writing `__init__`. 

In [1]:
import torch 
from tqdm import tqdm
from itertools import product as p
import json
import numpy as np
import h5py
from os.path import basename, dirname
#import dask.array as da
import pickle
from var import fname2mname

# setup

In [2]:
def pvec(t):
    return t/t.sum(dim=-1, keepdim=True)

In [3]:
class A:
    pass
self = A()

In [4]:
self.num_heads_d = {} # {fname, int}
self.attentions_d = {} # {fname, tensor}
f1, f2, f3 = "foo", "bar", "baz"
attention_fname_l = [f1, f2, f3]

In [5]:
# initialize `num_heads_d`, `attentions_d` with fake data
n1, n2, n3 = 10, 12, 14
wlen_l = [12, 5, 9, 6]

self.num_heads_d[f1] = n1
self.num_heads_d[f2] = n2
self.num_heads_d[f3] = n3

for fname in attention_fname_l:
    attentions_l = [pvec(torch.randn(self.num_heads_d[fname], wlen, wlen))
                        for wlen in wlen_l]
    self.attentions_d[fname] = attentions_l

In [6]:
self.device = torch.device('cpu')

# hnb

In [33]:
# set `self.representations_d` loop
self.representations_d = {}

In [12]:
# arbitrary values of loop vars
network = f1
al = self.attentions_d[network]

In [23]:
fal = [torch.flatten(at, start_dim=1).t()
          for at in al]

In [29]:
self.representations_d[network] = torch.cat(fal)

In [ ]:
# full loop
for network, al in self.attentions_d.items():
    fal = [torch.flatten(at, start_dim=1).t() for at in al]
    self.representations_d[network] = torch.cat(fal)